In [2]:
# Data visualization
import numpy as np
import pandas as pd 

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, TensorBoard
import keras_tuner as kt
from sklearn.metrics import accuracy_score
from keras.optimizers import RMSprop


# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

In [31]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE"
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [32]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

In [33]:
# load dataset
df = describe_dataset("./train.csv")

# Categorizing label
df.loc[df["label"] == "up", "label"] = 0

df.loc[df["label"] == "down", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v']
Number of rows: 4160 
Number of columns: 37

Labels: 
down    2127
up      2033
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [34]:
with open("./model/input_scaler.pkl", "rb") as f:
    sc = pickle.load(f)

In [35]:
# Standard Scaling of features
x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.fit_transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

In [37]:
stop_early = EarlyStopping(monitor='val_loss', patience=3)

# Final Results
final_models = {}

In [38]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")


def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer", "epoch"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

In [11]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model


In [13]:
tuner_3l = kt.Hyperband(
    model_3l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo',
)
tuner_3l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 02s]
val_accuracy: 0.9963942170143127

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 00m 35s
INFO:tensorflow:Oracle triggered exit


In [14]:
model_3l = get_best_model(tuner_3l)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x00000153085C4A60>
Layer-2: 320 units, func:  <function relu at 0x00000153085C4A60>
Layer-3: 2 units, func:  <function softmax at 0x00000153085C4040>

Other params:
learning_rate: 0.001
Epoch 1/100
333/333 [==============================] - 1s 1ms/step - loss: 0.0366 - accuracy: 0.9874 - val_loss: 0.0055 - val_accuracy: 0.9976
Epoch 2/100
333/333 [==============================] - 0s 1ms/step - loss: 0.0035 - accuracy: 0.9991 - val_loss: 0.0072 - val_accuracy: 0.9976
Epoch 3/100
333/333 [==============================] - 0s 1ms/step - loss: 0.0021 - accuracy: 0.9991 - val_loss: 0.0196 - val_accuracy: 0.9976
Epoch 4/100
333/333 [==============================] - 0s 1ms/step - loss: 0.0019 - accuracy: 0.9991 - val_loss: 0.0085 - val_accuracy: 0.9976


In [15]:
model_3l.save("./model/3LayeredModel.h5")

In [16]:
final_models["3_layers"] = model_3l

In [25]:
def model_5l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [26]:
tuner_5l = kt.Hyperband(
    model_5l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_2'
)
tuner_5l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 02s]
val_accuracy: 0.9987980723381042

Best val_accuracy So Far: 0.9987980723381042
Total elapsed time: 00h 00m 50s
INFO:tensorflow:Oracle triggered exit


In [27]:
model_5l = get_best_model(tuner_5l)
model_5l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x00000153085C4A60>
Layer-2: 160 units, func:  <function relu at 0x00000153085C4A60>
Layer-3: 384 units, func:  <function relu at 0x00000153085C4A60>
Layer-4: 480 units, func:  <function relu at 0x00000153085C4A60>
Layer-5: 2 units, func:  <function softmax at 0x00000153085C4040>

Other params:
learning_rate: 0.01
Epoch 1/100
333/333 [==============================] - 1s 2ms/step - loss: 0.1244 - accuracy: 0.9886 - val_loss: 0.0637 - val_accuracy: 0.9928
Epoch 2/100
333/333 [==============================] - 1s 2ms/step - loss: 0.0990 - accuracy: 0.9892 - val_loss: 0.3751 - val_accuracy: 0.9976
Epoch 3/100
333/333 [==============================] - 1s 2ms/step - loss: 0.0972 - accuracy: 0.9970 - val_loss: 0.0164 - val_accuracy: 0.9988
Epoch 4/100
333/333 [==============================] - 1s 2ms/step - loss: 4.0686e-04 - accuracy: 1.0000 - val_loss: 0.0539 - val_accuracy: 0.9988
Epoch 5/100
333/333 [==============

In [28]:
final_models["5_layers"] = model_5l

In [29]:
model_5l.save("./model/5LayeredModel.h5")

In [30]:
with open("./model/all_models.pkl", "wb") as f:
    pickle.dump(final_models, f)

In [39]:
def hybrid_model_builder(hp):
    model = Sequential()
    model.add(Dense(72, input_dim=36, activation='relu')) # Increase the number of neurons in the first layer
    
    # Add two additional dense layers
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    
    # Use a different optimizer and add dropout regularization
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop'])
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    
    if hp_optimizer == 'adam':
        optimizer = Adam(learning_rate=hp_learning_rate)
    else:
        optimizer = RMSprop(learning_rate=hp_learning_rate)
    
    model.add(Dropout(rate=hp_dropout))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [40]:
tuner_hybrid = kt.Hyperband(
    hybrid_model_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='hybrid_model_tuner'
)

tuner_hybrid.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard('./keras_tuner_dir/logs')])


Trial 30 Complete [00h 00m 05s]
val_accuracy: 0.9975961446762085

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 01m 21s
INFO:tensorflow:Oracle triggered exit


In [42]:
best_hybrid_model = get_best_model(tuner_hybrid)
best_hybrid_model.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])


Describe models architecture
Layer-1: 72 units, func:  <function relu at 0x00000153085C4A60>
Layer-2: 384 units, func:  <function relu at 0x00000153085C4A60>
Layer-3: 224 units, func:  <function relu at 0x00000153085C4A60>
Layer-4: 256 units, func:  <function relu at 0x00000153085C4A60>
Layer-5: 320 units, func:  <function relu at 0x00000153085C4A60>
Layer-6: 480 units, func:  <function relu at 0x00000153085C4A60>
Layer-7: 0 units, func: None
Layer-8: 2 units, func:  <function softmax at 0x00000153085C4040>

Other params:
learning_rate: 0.01
optimizer: adam
dropout: 0.1
Epoch 1/100
333/333 [==============================] - 1s 3ms/step - loss: 0.6375 - accuracy: 0.9871 - val_loss: 0.0192 - val_accuracy: 0.9952
Epoch 2/100
333/333 [==============================] - 1s 2ms/step - loss: 0.0239 - accuracy: 0.9958 - val_loss: 0.0263 - val_accuracy: 0.9964
Epoch 3/100
333/333 [==============================] - 1s 3ms/step - loss: 1.9894 - accuracy: 0.9832 - val_loss: 3.4738 - val_accuracy: 0

In [50]:
best_hybrid_model.save("./model/hybrid_model.h5")